In [ ]:
import requests
import time

# Initialization 
API_KEY = {'X-API-key': 'HCA61YVQ'}
s = requests.Session() 
s.headers.update(API_KEY)

resp = s.get('http://localhost:9999/v1/case')  # removed space before get
resp  # 200 means success

<Response [200]>

In [ ]:
def get_tick():
    """Get current tick of the case"""
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        return resp.json()['tick']
    return 0

def get_position(ticker):
    """Get current position for a ticker"""
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        for sec in resp.json():
            if sec['ticker'] == ticker:
                return sec['position']
    return 0

def cancel_all_orders():
    """Cancel all open orders"""
    resp = s.post('http://localhost:9999/v1/commands/cancel', 
                  params={'all': 1})
    if resp.ok:
        print('Cancelled all orders')

def send_order(ticker, action, price, quantity):
    """Send a limit order"""
    params = {
        'ticker': ticker,
        'type': 'LIMIT',
        'quantity': quantity,
        'action': action,
        'price': price
    }
    resp = s.post('http://localhost:9999/v1/orders', params=params)
    if resp.ok:
        order_id = resp.json()['order_id']
        print(f'SENT: {action} {ticker} {quantity} @ {price} (ID: {order_id})')
        return order_id
    else:
        print(f'FAILED: {resp.text}')
        return None

def market_make_appl():
    """Post wide markets on APPL"""
    # Cancel existing orders first
    cancel_all_orders()
    
    # Post super wide bid/ask
    send_order('APPL', 'BUY', 0, 100)     # bid at 0
    send_order('APPL', 'SELL', 1000, 100) # offer at 1000
    
    # Check position
    pos = get_position('APPL')
    print(f'Current APPL position: {pos}\n')

In [13]:
# test that the functions work.
import requests
import time

# Setup
API_KEY = {'X-API-key': 'HCA61YVQ'}
s = requests.Session()
s.headers.update(API_KEY)

print("="*50)
print("TESTING EACH FUNCTION")
print("="*50)

# TEST 1: Connection
print("\n1. Testing connection...")
resp = s.get('http://localhost:9999/v1/case')
print(f"Response status: {resp.status_code}")
print(f"Response OK? {resp.ok}")
if resp.ok:
    print(f"Case data: {resp.json()}")

# TEST 2: Get Tick
print("\n2. Testing get_tick()...")
def get_tick():
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        return resp.json()['tick']
    return 0

tick = get_tick()
print(f"Current tick: {tick}")

# TEST 3: Get all securities
print("\n3. Testing get securities...")
resp = s.get('http://localhost:9999/v1/securities')
print(f"Response OK? {resp.ok}")
if resp.ok:
    securities = resp.json()
    print(f"All securities: {securities}")

# TEST 4: Get APPL position
print("\n4. Testing get_position('APPL')...")
def get_position(ticker):
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        for sec in resp.json():
            if sec['ticker'] == ticker:
                return sec['position']
    return 0

pos = get_position('APPL')
print(f"APPL position: {pos}")

# TEST 5: Get order book
print("\n5. Testing get order book for APPL...")
resp = s.get('http://localhost:9999/v1/securities/book',
             params={'ticker': 'APPL'})
print(f"Response OK? {resp.ok}")
if resp.ok:
    book = resp.json()
    print(f"Bids: {book['bids'][:3] if book['bids'] else 'EMPTY'}")
    print(f"Asks: {book['asks'][:3] if book['asks'] else 'EMPTY'}")

# TEST 6: Send a test order
print("\n6. Testing send_order()...")
test = input("Send test BUY order for 1 share at $10? (y/n): ")
if test.lower() == 'y':
    def send_order(ticker, action, price, qty):
        params = {
            'ticker': ticker,
            'type': 'LIMIT',
            'quantity': qty,
            'action': action,
            'price': price
        }
        resp = s.post('http://localhost:9999/v1/orders', params=params)
        if resp.ok:
            order_id = resp.json()['order_id']
            print(f'SUCCESS: {action} {qty} {ticker} @ {price} (ID: {order_id})')
            return order_id
        else:
            print(f'FAILED: {resp.text}')
            return None
    
    order_id = send_order('APPL', 'BUY', 10, 1)

# TEST 7: Check open orders
print("\n7. Testing get open orders...")
resp = s.get('http://localhost:9999/v1/orders')
print(f"Response OK? {resp.ok}")
if resp.ok:
    orders = resp.json()
    print(f"Open orders: {orders}")

# TEST 8: Cancel all orders
print("\n8. Testing cancel_all_orders()...")
test2 = input("Cancel all open orders? (y/n): ")
if test2.lower() == 'y':
    def cancel_all_orders():
        resp = s.post('http://localhost:9999/v1/commands/cancel',
                      params={'all': 1})
        if resp.ok:
            result = resp.json()
            print(f'Cancelled orders: {result}')
            return True
        else:
            print(f'Failed: {resp.text}')
            return False
    
    cancel_all_orders()

print("\n" + "="*50)
print("TESTING COMPLETE!")
print("="*50)

TESTING EACH FUNCTION

1. Testing connection...
Response status: 200
Response OK? True
Case data: {'name': 'Farming Trading Simulation', 'period': 1, 'tick': 149, 'ticks_per_period': 900, 'total_periods': 1, 'status': 'ACTIVE', 'is_enforce_trading_limits': False}

2. Testing get_tick()...
Current tick: 149

3. Testing get securities...
Response OK? True
All securities: [{'ticker': 'APPL', 'type': 'STOCK', 'size': 1, 'position': 0.0, 'vwap': 0.0, 'nlv': 0.0, 'last': -100.0, 'bid': 0.0, 'bid_size': 0.0, 'ask': -100.0, 'ask_size': 3100.0, 'volume': 28200.0, 'unrealized': 0.0, 'realized': 0.0, 'currency': '', 'total_volume': 28200.0, 'limits': [{'name': 'LIMIT-STOCK', 'units': 1.0}], 'interest_rate': 0.0, 'is_tradeable': True, 'is_shortable': True, 'start_period': 1, 'stop_period': 1, 'description': 'APPL will settle to the collective harvest of apples', 'unit_multiplier': 1, 'display_unit': '', 'start_price': 0.0, 'min_price': -100.0, 'max_price': 1100.0, 'quoted_decimals': 2, 'trading_fe

In [11]:
print('Starting APPL market maker...')

cancel_all_orders()
send_order('APPL', 'BUY', -100, 100)
send_order('APPL', 'SELL', 1000, 100)
print('Orders posted!\n')

shutdown = False
while not shutdown:
    tick = get_tick()
    
    if tick > 895:
        print('Case ending soon, stopping...')
        shutdown = True
        break
    
    # Just monitor position
    pos = get_position('APPL')
    print(f'Tick: {tick}, Position: {pos}')
    
    time.sleep(5)  # Check every 5 seconds

# Cleanup
cancel_all_orders()
print('Done!')

Starting APPL market maker...
Cancelled all orders
SENT: BUY APPL 100 @ -100 (ID: 14122)
SENT: SELL APPL 100 @ 1000 (ID: 14123)
Orders posted!

Tick: 848, Position: -1290.0
Tick: 853, Position: -1190.0
Tick: 858, Position: -1190.0
Tick: 863, Position: -1190.0
Tick: 868, Position: -1190.0
Tick: 873, Position: -1190.0
Tick: 878, Position: -1190.0
Tick: 883, Position: -1190.0
Tick: 888, Position: -1190.0
Tick: 893, Position: -1190.0
Case ending soon, stopping...
Cancelled all orders
Done!
